In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [3]:
from costs import *
from proj1_helpers import *
from franksun_function import *
from helpers import *

In [4]:
def build_poly(x, degree):
    poly_row = lambda x: [x**j for j in range(degree + 1)]
    return np.array([poly_row(e) for e in x])

In [26]:
def standardize(x, y):
    cols = [4, 5, 6, 12, 26, 27, 28]
    
    # Get the column where the above columns are not -999
    index = []
    for i in range(len(x)):
        for col in cols:
            if x[i][col] != -999:
                index.append(i)
                break
    
    # Get rid of those columns
    std_tX = np.delete(x, index, axis=0)
    std_y = np.delete(y, index)
    return std_tX, std_y

In [29]:
std_tX, std_y = standardize(tX, y)
print("finished standardization")


lambdas = np.logspace(-4, 2, 20)

for lambda_ in lambdas:
#     print(std_tX.shape)
#     print(std_y.shape)
    weights = ridge_regression(std_y, std_tX, lambda_)
    print(weights)
    print("Lamda: ", lambda_, " Performance: ", performance(weights, y, tX))



finished standardization
[  1.92390025e-05  -7.78828754e-03  -7.97376035e-03  -2.89361467e-03
  -3.08075978e-02  -3.10901295e-02  -2.97786165e-02   4.04812919e-01
   1.17636459e-03  -2.11798736e+00  -2.45321553e-01   7.75543682e-02
  -8.20813680e-02   2.12859683e+00   8.04757946e-04  -9.59667054e-04
   2.13236253e+00   5.53235129e-04   7.21929932e-04   3.55687014e-03
  -5.96569367e-04   4.74119761e-05   2.39084843e-01   2.49276725e-03
  -9.27165008e-04  -1.79755343e-03   1.05294978e-01   1.05294978e-01
  -3.54926201e-02   2.11786033e+00]
Lamda:  0.0001  Performance:  0.65056
[  1.92389369e-05  -7.78829010e-03  -7.97376074e-03  -2.89361390e-03
  -1.83574992e-02  -3.12591947e-02  -1.97662979e-02   4.04812930e-01
   1.17636510e-03  -2.10034607e+00  -2.45321550e-01   7.75543395e-02
  -2.35381236e-02   2.11095554e+00   8.04757839e-04  -9.59665830e-04
   2.11472124e+00   5.53252493e-04   7.21928655e-04   3.55687138e-03
  -5.96561569e-04   4.74120723e-05   1.14514428e-01   2.61746774e-03
  -9

## Generate predictions and save ouput in csv format for submission:

In [5]:
DATA_TEST_PATH = '../../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [9]:
OUTPUT_PATH = '../../data/output.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)